In [1]:
import re
import pandas as pd
import numpy as np
import pickle

pd.set_option("display.max_colwidth", -1)
import codecs

import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
Pstemmer = nltk.stem.PorterStemmer()
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()


import string

import gensim
from gensim import corpora, models, similarities
from gensim.utils import lemmatize
from gensim.models.coherencemodel import CoherenceModel

from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []

    def handle_data(self, d):
        self.fed.append(d)

    def get_data(self):
        return " ".join(self.fed)


def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 8] nodename nor servname provided, or not
[nltk_data]     known>
[nltk_data] Error loading wordnet: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


In [2]:
dog_data_SF_19_full = pd.read_csv(
    "/Users/allisonkelly/Documents/Insight/Petfinder_Project/Model_Data/dog_data_SF_19_full.csv"
)

In [3]:
dog_data_SF_19_full.columns

Index(['animal_id', 'age', 'attributes.house_trained',
       'attributes.shots_current', 'attributes.spayed_neutered',
       'attributes.special_needs', 'breeds.mixed', 'breeds.primary', 'coat',
       'colors.primary', 'contact.address.city', 'environment.cats',
       'environment.children', 'environment.dogs', 'gender', 'name',
       'organization_id.1', 'size', 'status', 'url', 'num_photo', 'pub_year',
       'pub_month', 'pub_day', 'missing_description', 'pub_time', 'stat_time',
       'time_in_shelter', 'time_adpt_med', 'tags_cln', 'tags_top', 'num_tags',
       'colors_cat', 'age_Adult', 'age_Baby', 'age_Senior', 'age_Young',
       'gender_Female', 'gender_Male', 'size_Extra Large', 'size_Large',
       'size_Medium', 'size_Small', 'coat_Curly', 'coat_Hairless', 'coat_Long',
       'coat_Medium', 'coat_Short', 'coat_Wire', 'colors_cat_ct_dark',
       'colors_cat_ct_light', 'colors_cat_ct_med', 'pub_month_4',
       'pub_month_5', 'pub_month_6', 'pub_month_7', 'pub_month_8',

### Cleaning up text data

In [4]:
def cases_clean(text_str):
    if pd.isnull(text_str) != True:
        testc1 = strip_tags(text_str)
        testc2 = re.sub(
            ".?(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)*\/?", "", testc1
        )
        testc3 = testc2.strip(string.punctuation)
        textc4 = re.sub("\s+", " ", testc3)
        textc5 = re.sub("[^A-Za-z0-9]+", " ", textc4)
        textc6 = re.sub("\d+", " ", textc5)
        return textc6

In [5]:
dog_data_SF_19_full["desc_full_clean"] = dog_data_SF_19_full["full_description"].apply(
    cases_clean
)

In [6]:
dog_data_SF_19_full["desc_full_clean"][200:202]

200     Buck is here and ready to meet you This wonderful lab mix is mellow and sweet At about   years old Buck is ready for a relaxed life He enjoys going for walks and playing but he also likes relaxing and peacefully snoozing In Buck s previous home he lived outdoor only so he will need some help transitioning to an indoor lifestyle He s hoping his new family will be patient with him and allow him the time he needs Buck is uncomfortable with invasive handling so he s looking for a home with just teens and adults that will handle him gently Buck enjoys the company of other dogs and has a good history with cats He is not a fit for an apartment or condo Come meet Buck today Adoption hours Wednesday Sunday Noon  pm We begin accepting applications at  am each adoption day and will help people in the order of their arrival beginning at Noon We ask that all household members including any resident dog s be here to meet an animal prior to adoption We are unable to place animals on hold or t

### Create a length variable

In [11]:
def extract_len(text_str):
    if pd.isnull(text_str) != True:
        return len(text_str)

In [12]:
dog_data_SF_19_full["len_descr"] = dog_data_SF_19_full["desc_full_clean"].apply(
    extract_len
)

In [13]:
dog_data_SF_19_full["len_descr"].describe()

count    3994.000000
mean     1134.665248
std      629.398717 
min      15.000000  
25%      678.000000 
50%      1062.000000
75%      1506.500000
max      6228.000000
Name: len_descr, dtype: float64

In [14]:
dog_data_SF_19_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3995 entries, 0 to 3994
Data columns (total 68 columns):
animal_id                     3995 non-null int64
age                           3995 non-null object
attributes.house_trained      3995 non-null int64
attributes.shots_current      3995 non-null int64
attributes.spayed_neutered    3995 non-null int64
attributes.special_needs      3995 non-null int64
breeds.mixed                  3995 non-null int64
breeds.primary                3995 non-null object
coat                          2615 non-null object
colors.primary                2291 non-null object
contact.address.city          3995 non-null object
environment.cats              1590 non-null object
environment.children          2455 non-null object
environment.dogs              3160 non-null object
gender                        3995 non-null object
name                          3995 non-null object
organization_id.1             3995 non-null object
size                          39

In [15]:
dog_data_SF_19_full.to_csv(
    "/Users/allisonkelly/Documents/Insight/Petfinder_Project/Model_Data/dog_data_SF_ready.csv", index=False
)